In [1]:
# Importar librerias y configurar logging
import pandas as pd
import numpy as np
from pathlib import Path
import tabula
from unidecode import unidecode
import logging
logging.basicConfig(level=logging.DEBUG, filename='log.log',  format='%(asctime)s %(levelname)s %(name)s %(message)s')
logger = logging.getLogger(__name__)

In [2]:
# Listar archivos pdf
mi_path = 'fichas_simp/'
lista_pdfs = list(Path(mi_path).glob('**/*.pdf'))
nombres_pdfs = [pdf.name.replace('.pdf', '') for pdf in lista_pdfs]

In [3]:
# Mis funciones
def remove_header(df):
    return df.T.reset_index().T.reset_index(drop=True)
def get_indexes(list_dfs):
    list_adjudicado = []
    list_item = []
    list_cronograma = []
    idx_adjudicado = None
    idx_item = None
    idx_cronograma = None
    for i in range(len(list_dfs)):
        if list_dfs[i].columns.str.contains('Nombre proponente').any():
            list_adjudicado.append(i)
        if list_dfs[i].columns.str.contains('UNSPS').any():
            list_item.append(i)
        if list_dfs[i].columns.str.contains('Hora').any():
            list_cronograma.append(i)
    idx_adjudicado = list_adjudicado[0] if len(list_adjudicado) > 0 else None
    idx_item = list_item[0] if len(list_item) > 0 else None
    idx_cronograma = list_cronograma[0] if len(list_cronograma) > 0 else idx_item + 1
    return idx_adjudicado, idx_item, idx_cronograma

In [ ]:
# Extraer tablas
tablas = []
for pdf in lista_pdfs:
    try:
        tabla = tabula.read_pdf(pdf, pages='all', guess=False, lattice=True)
        tablas.append(tabla)
    except Exception as e:
        tablas.append(None)
        logger.error(f'Error al extraer tablas de {pdf}')
        logger.error(e)
        continue

In [4]:
# read pickle file
tablas =  pd.read_pickle('fichas.pkl')

In [10]:
del tablas

In [5]:
# Obtener tablas
tablas_item = []
tablas_adjudicado = []
for idx, tabla in enumerate(tablas):
    try:
        [idx_adjudicado, idx_item, idx_cronograma] = get_indexes(tabla)
        tabla_item = [remove_header(x) for x in tabla[idx_item:idx_cronograma]]
        # tabla_item.columns = tabla_item.columns.str.replace('\r', '', regex=True).str.replace(' ', '').to_series().apply(unidecode)
        for df in tabla_item:
            df.drop(df.columns[-1], axis=1, inplace=True)
            # df.drop(df.index[-1], inplace=True)
            df.replace('\r', ' ', regex=True, inplace=True)
            df.insert(0, 'cuce', nombres_pdfs[idx])
            df.insert(1, 'list_n', idx)
            tablas_item.append(df)
    except Exception as e:
        tablas_item.append(None)
        logger.error(f'Error al procesar tablas de ITEM de la lista: {idx}: {nombres_pdfs[idx]}')
        logger.error(e)
        continue
    try:
        tabla_adjudicado = tabla[idx_adjudicado:idx_item]
        for df in tabla_adjudicado:
            df.insert(0, 'cuce', nombres_pdfs[idx])
            df.insert(1, 'list_n', idx)
            df.replace('\r', ' ', regex=True, inplace=True)
            tablas_adjudicado.append(df)
    except Exception as e:
        tablas_adjudicado.append(None)
        logger.error(f'Error al procesar tablas de ADJUDICADO de la lista: {idx}: {nombres_pdfs[idx]}')
        logger.error(e)
        continue

In [18]:
# limpiando tablas item
nombres_cols = ['cuce', 'list_n', 'numero','codigo_unspsc','objeto_de_gasto','descripcion_del_bien_o_servicio','unidad_de_medida','cantidad','precio_referencial_unitario','precio_referencial_total','nombre_o_razon_social_del_proponente_adjudicado','precio_unitario_adjudicado','total_adjudicado','cantidad_recepcionada','precio_unitario_real','monto_real_ejecutado','estado','cronograma']
tablas_item_clean = pd.concat(tablas_item, ignore_index=True)
tablas_item_clean.columns = nombres_cols
tablas_item_clean.query('numero != "#"', inplace=True)
tablas_item_clean.query('numero not in ["Total Adjudicado:", "Total Referencial:"]', inplace=True)
tablas_item_clean.query('numero != "Unnamed: 0"', inplace=True)

tablas_item_clean.to_csv('tablas_clean_item.csv', index=False)


In [8]:
# limpiando tablas adjudicado
tablas_adjudicado_clean = pd.concat(tablas_adjudicado, ignore_index=True)
tablas_adjudicado_clean.to_csv('tablas_clean_adjudicado.csv', index=False)

In [9]:
# limpiando tablas totales
tablas_totales = tablas_item_clean.query('numero == "Total Adjudicado:" | numero == "Total Referencial:"').iloc[:, 0:8]
tablas_totales.columns = ['cuce', 'list_n', 'tipo_a', 'monto_a', 'tipo_b', 'monto_b', 'tipo_c', 'monto_c']

tablas_totales.to_csv('tablas_totales.csv', index=False)

In [ ]:
import pickle
with open('fichas.pkl', 'wb') as f:
    pickle.dump(tablas, f)

with open('lista_pdfs.pkl', 'wb') as f:
    pickle.dump(lista_pdfs, f)

In [ ]:
len(lista_pdfs) == len(tablas) == len(tablas_clean)

In [14]:
lista_pdfs[9]

PosixPath('fichas_simp/plasma/20-1810-00-1049162-0-E.pdf')

# Pruebas

In [19]:
lista_pdfs[359]

PosixPath('fichas_simp/PCR/21-0417-07-1121399-1-1.pdf')

In [20]:
tablas[359][5]

,20,4112000,34200,CLORO COLORIMETRICO,CAJA,1.00,220.00,220.00.1,Unnamed: 0,Unnamed: 1,Unnamed: 2,1.00.1,218.00,218.00.1,Contratado,Cronograma 0
0,21,4112000,34200,PRUEBA RAPIDA PARA PSA\rX 25 UNIDADES,CAJA,3.0,550.0,"1,650.00",NaN,NaN,NaN,3.0,548.0,"1,644.00",Contratado,Cronograma 0
1,22,4112000,34200,FIBRINOGENO 1 X 10 ML,CAJA,4.0,607.0,"2,428.00",NaN,NaN,NaN,4.0,605.0,"2,420.00",Contratado,Cronograma 0
2,23,4112000,34200,PRUEBA RAPIDA PARA\rCHAGAS X 25 UNIDADES,CAJA,1.0,625.0,625.00,NaN,NaN,NaN,1.0,622.0,622.00,Contratado,Cronograma 0
3,Total Referencial:,"49,838.00",Total Recepcionado:,"49,312.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
get_indexes(tablas[9])

(3, 4, 5)